# Phillies Quantitative Analyst Take-Home
### Author: Ryan Williams
### Date: 05/12/2025

**Language**: Julia 1.11+

This notebook addresses Question 11 of the assessment, where we are tasked with predicting each pitcher's 2024 strikeout percentage (K%) using their prior performance data.

We use a hierarchical Bayesian model implemented via `Turing.jl` to estimate future K% while accounting for player-level effects and uncertainty.


In [ ]:
# Setup Julia environment for the k_model package
using Pkg
Pkg.activate("k_model_env")
Pkg.add([
    "CSV",
    "DataFrames",
    "StatsPlots",
    "Turing",
    "Random",
    "Distributions",
    "StatsBase",
    "CategoricalArrays",
    "MCMCChains",
    "MLJ",
    "MLJLinearModels"
])
Pkg.precompile()

  Activating project at `~/git/QuantAnalysisAssets/phillies_takehome/k_model_env`
   Resolving package versions...
   Installed PrettyPrint ────────────── v0.2.0
   Installed CategoricalDistributions ─ v0.1.15
   Installed ContextVariablesX ──────── v0.1.3
   Installed ShowCases ──────────────── v0.1.0
   Installed EarlyStopping ──────────── v0.3.0
   Installed LearnAPI ───────────────── v0.1.0
   Installed IterationControl ───────── v0.5.4
   Installed MLJModels ──────────────── v0.17.9
   Installed MLUtils ────────────────── v0.4.8
   Installed MLCore ─────────────────── v1.0.0
   Installed FLoopsBase ─────────────── v0.1.1
   Installed NameResolution ─────────── v0.1.5
   Installed PrettyPrinting ─────────── v0.4.2
   Installed ComputationalResources ─── v0.3.2
   Installed MLStyle ────────────────── v0.4.17
   Installed FLoops ─────────────────── v0.2.2
   Installed MLJFlow ────────────────── v0.5.0
   Installed StatisticalMeasures ────── v0.1.7
   Installed MLJBase ───────────────

In [13]:
# Load libraries
using CSV, DataFrames
using StatsPlots
using Turing
using Random, Distributions
using StatsBase
# using CategoricalArrays
using LinearAlgebra
using MCMCChains
using MLJ
using MLJLinearModels
Random.seed!(42)

ArgumentError: ArgumentError: Package MLJLinearModels not found in current path.
- Run `import Pkg; Pkg.add("MLJLinearModels")` to install the MLJLinearModels package.

## Load and inspect data
We begin by loading `k.csv`, which contains:
- Player identifiers (MLBAMID, FanGraphs ID)
- Age and season
- Total Batters Faced (TBF) and Strikeout Percentage (K%)

In [3]:
df = CSV.read("k.csv", DataFrame)
first(df, 5)

Row,MLBAMID,PlayerId,Name,Team,Age,Season,TBF,K%
,Int64,Int64,String31,String7,Int64,Int64,Int64,Float64
1,695243,31757,Mason Miller,OAK,25,2024,249,0.417671
2,621242,14710,Edwin Díaz,NYM,30,2024,216,0.388889
3,518585,7048,Fernando Cruz,CIN,34,2024,288,0.378472
4,623352,14212,Josh Hader,HOU,30,2024,278,0.377698
5,663574,19926,Tony Santillan,CIN,27,2024,122,0.377049


In [4]:
describe(df)
# names(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,MLBAMID,6.20025e5,425794,641712.0,808967,0,Int64
2,PlayerId,17062.8,1246,17485.0,33876,0,Int64
3,Name,,A.J. Minter,,Zebby Matthews,0,String31
4,Team,,- - -,,WSN,0,String7
5,Age,28.7405,20,28.0,43,0,Int64
6,Season,2022.5,2021,2023.0,2024,0,Int64
7,TBF,344.901,115,268.0,886,0,Int64
8,K%,0.232229,0.0914634,0.22836,0.502128,0,Float64


## Preprocess Data
- We sort by season and player
- We compute lagged K% and TBF
- We exclude rows with missing historical data (e.g., rookies)

In [5]:
rename!(df, Symbol("K%") => :K)

Row,MLBAMID,PlayerId,Name,Team,Age,Season,TBF,K
,Int64,Int64,String31,String7,Int64,Int64,Int64,Float64
1,695243,31757,Mason Miller,OAK,25,2024,249,0.417671
2,621242,14710,Edwin Díaz,NYM,30,2024,216,0.388889
3,518585,7048,Fernando Cruz,CIN,34,2024,288,0.378472
4,623352,14212,Josh Hader,HOU,30,2024,278,0.377698
5,663574,19926,Tony Santillan,CIN,27,2024,122,0.377049
6,669093,22210,Jeremiah Estrada,SDP,25,2024,252,0.373016
7,547973,10233,Aroldis Chapman,PIT,36,2024,265,0.369811
8,671305,22533,Michel Otañez,OAK,26,2024,151,0.364238
9,489446,9073,Kirby Yates,TEX,37,2024,237,0.35865


In [7]:
# Sort the DataFrame to ensure proper ordering within each player group
sort!(df, [:MLBAMID, :Season])

# Initialize new columns with missing values
df[!, :K_prev] = Vector{Union{Missing, Float64}}(missing, nrow(df))
df[!, :TBF_prev] = Vector{Union{Missing, Int64}}(missing, nrow(df))

# Fill previous season's K% and TBF for each pitcher
for g in groupby(df, :MLBAMID)
    global_indices = findall(x -> x in g.MLBAMID, df.MLBAMID)
    for i in 2:nrow(g)
        curr_idx = global_indices[i]
        prev_idx = global_indices[i - 1]

        df[!, :K_prev][curr_idx] = df[!, :K][prev_idx]
        df[!, :TBF_prev][curr_idx] = df[!, :TBF][prev_idx]
    end
end

# Drop rows that don’t have lagged values (e.g. rookies or first-year records)
df_model = dropmissing(df, [:K_prev, :TBF_prev])

# Add derived features
df_model[!, :ΔK] = df_model.K .- df_model.K_prev
df_model[!, :is_relief] = df_model.TBF .< 200

1029-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

### Scatter Plot of Features

In [6]:
@df df_model scatter(
    :TBF_prev, :ΔK,
    xlabel = "TBF (Previous Season)",
    ylabel = "Change in K%",
    title = "TBF vs. ΔK — Pitcher Improvement by Workload",
    alpha = 0.6,
    legend = false
)

UndefVarError: UndefVarError: `df_model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Target and Features
- Target: K% for current season
- Features: prior K%, prior TBF, age

In [8]:
# Step 1: Create unique list of players
unique_players = unique(df_model.MLBAMID)

# Step 2: Create a lookup table mapping each player ID to an index
player_lookup = Dict(pid => i for (i, pid) in enumerate(unique_players))

# Step 3: Create player_index by mapping each row's MLBAMID to its index
player_index = [player_lookup[pid] for pid in df_model.MLBAMID]

# X features to use for now
X = select(df_model, [:K_prev, :TBF_prev, :Age]) |> Matrix

# y target
y = df_model.K |> collect

# player_index is now ready
# -> use it in: θ[player_index]

1029-element Vector{Float64}:
 0.17808219
 0.11363636
 0.12478632
 0.16357504
 0.21524664
 0.18686869
 0.32692308
 0.27659574
 0.28440367
 0.25342466
 ⋮
 0.22180451
 0.21398305
 0.17486339
 0.20353982
 0.24342105
 0.17445483
 0.2
 0.41767068
 0.19811321

In [9]:
X_df = select(df_model, [:K_prev, :TBF_prev, :Age])
y_vec = df_model.K

1029-element Vector{Float64}:
 0.17808219
 0.11363636
 0.12478632
 0.16357504
 0.21524664
 0.18686869
 0.32692308
 0.27659574
 0.28440367
 0.25342466
 ⋮
 0.22180451
 0.21398305
 0.17486339
 0.20353982
 0.24342105
 0.17445483
 0.2
 0.41767068
 0.19811321

In [10]:
model = RidgeRegressor(lambda=0.1)  # You can tune this
mach = machine(model, X_df, y_vec)
fit!(mach)

UndefVarError: UndefVarError: `RidgeRegressor` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
y_pred = predict(mach, X_df)  # Returns vector of predicted K%
df_model.K_pred = y_pred

## Turing Model Definition
We model K% as a function of prior stats, with player-level intercepts.

In [71]:
@model function k_predict_model(X, y, player_idx)
    N, D = size(X)
    n_players = maximum(player_idx)

    # μ = predicted K%
    # β = linear model coefficients
    # σ = residual stddev
    # θ = player-specific effect

    α ~ Normal(0, 1)
    β ~ MvNormal(D, 1.0)
    σ ~ Exponential(1.0)
    θ_player ~ filldist(Normal(0, 1), n_players)

    for i in 1:N
        μ = α + θ_player[player_idx[i]] + dot(X[i, :], β)
        y[i] ~ Normal(μ, σ)
    end
end

k_predict_model (generic function with 2 methods)

## Fit the Model

In [10]:
using Base.Threads
@show nthreads()

nthreads() = 4


4

In [11]:
Threads.@threads for i in 1:Threads.nthreads()
    println("Thread $i alive")
end

Thread 1 alive
Thread 3 alive
Thread 2 alive
Thread 4 alive


In [ ]:
model = k_predict_model(X, y, player_index)
sampler = NUTS(adtype = AutoForwardDiff())
chain = sample(model, sampler, MCMCThreads(), 1000, Threads.nthreads(); progress=true)

Sampling (4 threads)   0%|                              |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference /Users/ryanwilliams/.julia/packages/Turing/nQwbh/src/mcmc/hmc.jl:207
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference /Users/ryanwilliams/.julia/packages/Turing/nQwbh/src/mcmc/hmc.jl:207
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference /Users/ryanwilliams/.julia/packages/Turing/nQwbh/src/mcmc/hmc.jl:207
┌ Info: Found initial step size
│   ϵ = 4.8828125e-5
└ @ Turing.Inference /Users/ryanwilliams/.julia/packages/Turing/nQwbh/src/mcmc/hmc.jl:207


## Posterior Check (optional)